In [1]:
import httpx
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores.pgvector import PGVector

print("Loading environment variables...")
# Load environment variables from .env file
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")
print("Environment variables loaded.")

# --- SSL FIX FOR CORPORATE PROXY ---
print("Creating insecure httpx client...")
insecure_client = httpx.Client(verify=False)
print("Client created.")

# --- Initialize Clients ---
print("Initializing LLM and Embeddings clients...")
llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY, http_client=insecure_client)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=OPENAI_API_KEY, http_client=insecure_client)
print("Clients initialized.")

Loading environment variables...
Environment variables loaded.
Creating insecure httpx client...
Client created.
Initializing LLM and Embeddings clients...
Clients initialized.


In [2]:
question = "What is the main purpose of the system described in the document?"
print(f"Question set to: '{question}'")

Question set to: 'What is the main purpose of the system described in the document?'


In [3]:
COLLECTION_NAME = "quasar_doc_collection"

print("Initializing PGVector store...")
vector_store = PGVector(
    connection_string=DATABASE_URL,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
)

print("Creating retriever...")
retriever = vector_store.as_retriever()

print(f"Retrieving context for the question: '{question}'...")
retrieved_docs = retriever.invoke(question)

print("\n--- SUCCESS: CONTEXT RETRIEVED ---")
print(f"Retrieved {len(retrieved_docs)} document chunks.")
# Optional: print the retrieved context to see what the agent found
# for i, doc in enumerate(retrieved_docs):
#     print(f"\n--- Chunk {i+1} ---")
#     print(doc.page_content)

Initializing PGVector store...
Creating retriever...
Retrieving context for the question: 'What is the main purpose of the system described in the document?'...


/tmp/ipykernel_2162/3580055909.py:4: LangChainPendingDeprecationWarning: This class is pending deprecation and may be removed in a future version. You can swap to using the `PGVector` implementation in `langchain_postgres`. Please read the guidelines in the doc-string of this class to follow prior to migrating as there are some differences between the implementations. See <https://github.com/langchain-ai/langchain-postgres> for details about the new implementation.
  vector_store = PGVector(
/tmp/ipykernel_2162/3580055909.py:4: LangChainPendingDeprecationWarning: Please use JSONB instead of JSON for metadata. This change will allow for more efficient querying that involves filtering based on metadata. Please note that filtering operators have been changed when using JSONB metadata to be prefixed with a $ sign to avoid name collisions with columns. If you're using an existing database, you will need to create a db migration for your metadata column to be JSONB and update your queries to


--- SUCCESS: CONTEXT RETRIEVED ---
Retrieved 4 document chunks.


In [4]:
prompt_template = """
You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.

Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt_template)
rag_chain = prompt | llm | StrOutputParser()

print("--- GENERATING ANSWER (This is the test) ---")
print("Sending request to LLM... Please wait...")

# This is the line that is likely hanging
final_answer = rag_chain.invoke({"question": question, "context": retrieved_docs})

print("\n--- SUCCESS: ANSWER GENERATED! ---")
print("Final Answer:", final_answer)

--- GENERATING ANSWER (This is the test) ---
Sending request to LLM... Please wait...

--- SUCCESS: ANSWER GENERATED! ---
Final Answer: The main purpose of the system described in the document is the development and validation of a computational model for early Alzheimer's Disease and Related Disorders (ADRD) diagnosis.
